In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd 
import cv2
from keras.models import Model
import glob
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from scipy.signal.signaltools import wiener
import h5py
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization,Conv2D, SeparableConv2D, MaxPooling2D, LeakyReLU, Activation, GlobalMaxPooling2D
from keras.optimizers import Adam
import imgaug.augmenters as iaa
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from google_drive_downloader import GoogleDriveDownloader as gdd

In [ ]:
gdd.download_file_from_google_drive(file_id='1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY',dest_path='content/pneumonia_dataset.zip',unzip=True)

Unzipping...Done.


In [ ]:
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip = True, brightness_range=[0.7, 1.3]) 
img_dir = 'content/pneumonia_dataset/train/normal'  
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)

x = img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
path, dirs, files = next(os.walk("content/pneumonia_dataset/train/normal"))
file_count = len(files) 

for batch in datagen.flow (x, batch_size=1, save_to_dir =r'train/aug-normal',save_prefix="a",save_format='jpg'):
    i+=1
    if i==file_count*3:
        break

datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip = True, brightness_range=[0.7, 1.3]) 
img_dir = 'content/pneumonia_dataset/train/pneumonia'  
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)

x = img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
path, dirs, files = next(os.walk("content/pneumonia_dataset/train/pneumonia"))
file_count = len(files) 

for batch in datagen.flow (x, batch_size=1, save_to_dir =r'train/aug-pneumonia',save_prefix="a",save_format='jpg'):
    i+=1
    if i==file_count*3:
        break

NameError: ignored

In [ ]:
train_X = []
train_y = []
dim=(299,299)

img_folder_path='content/pneumonia_dataset/'
mode = 'train'
classes = os.listdir(os.path.join(img_folder_path, mode))
for cl in classes:
    files = os.listdir(os.path.join(img_folder_path, mode, cl))
    for f in files:
        image_path= os.path.join(img_folder_path,mode, cl, f)
        img=cv2.imread(image_path,0)
        img=img[20:980,20:980]
        equ = cv2.equalizeHist(img)
        img = cv2.resize(equ, dim)
        img = np.dstack([img, img, img])
        train_X.append(img)
        if cl=='normal':
             train_y.append(0)
        else:
             train_y.append(1)
        
train_X = np.asarray(train_X)
train_y = np.asarray(train_y)

In [ ]:
print(train_X.shape)
print(train_y.shape)

(2425, 299, 299, 3)
(2425,)


In [ ]:
from tensorflow.keras.applications.xception import Xception

input_shape = (299,299,3)

model = Sequential()
pretrained_model = Xception(input_shape = input_shape,  weights = 'imagenet',  include_top = False)
for layer in pretrained_model.layers[:40]:
      layer.trainable = False

for layer in pretrained_model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True
 

model.add(pretrained_model)

model.layers.pop()
model.layers.pop()
model.layers.pop()

model.add(GlobalMaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.75))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 20,865,578
Trainable params: 19,174,618
Non

In [ ]:
opt = Adam(lr=0.001, decay=1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

In [ ]:
train_y=train_y.reshape(-1,1)
train_y = OneHotEncoder().fit_transform(train_y).toarray()
print(train_y.shape)

(2425, 2)


In [ ]:
train_y2=train_y
train_y = np.asarray(train_y).astype('float32').reshape((-1,1))

In [ ]:
tf.config.run_functions_eagerly(False)

In [ ]:
model.fit(train_X, train_y2, verbose=1, epochs=4, callbacks = [es,chkpt])

Epoch 1/4
76/76 [==============================] - 3146s 41s/step - loss: 0.6829 - accuracy: 0.6227
Epoch 2/4
76/76 [==============================] - 3215s 42s/step - loss: 0.5981 - accuracy: 0.6905
Epoch 3/4
76/76 [==============================] - 3188s 42s/step - loss: 0.5617 - accuracy: 0.7237
Epoch 4/4
76/76 [==============================] - 3180s 42s/step - loss: 0.4793 - accuracy: 0.7754


In [ ]:
test_X = []
dim=(299,299)
img_folder_path='content/pneumonia_dataset/'

mode='test'
files = os.listdir(os.path.join(img_folder_path, mode))
for f in files:
        image_path= os.path.join(img_folder_path, mode, f)
        img=cv2.imread(image_path,0)
        img=img[20:980,20:980]
        equ = cv2.equalizeHist(img)
        img = cv2.resize(equ, dim)
        img = np.dstack([img, img, img])
        test_X.append(img)
        
test_X = np.asarray(test_X)
preds = model.predict(test_X)

In [ ]:
preds = model.predict(test_X)

In [ ]:
sol=[]
for i in range(len(preds)):
    if preds[i][0] > preds[i][1]:
        sol.append('normal')
    else:
        sol.append('pneumonia')

In [ ]:
print(sol)

['normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'normal', 'pneumonia', 'normal', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'normal', 'normal', 'normal', 'pneumonia', 'normal', 'pneumonia', 'pneumonia', 'pneumonia', 'normal', 'normal', 'pneumonia', 'pneumonia', 'normal', 'normal

In [ ]:
names_X = []
dim=(299,299)
img_folder_path='content/pneumonia_dataset/'

mode='test'
files = os.listdir(os.path.join(img_folder_path, mode))
for f in files:
      names_X.append(f)

sol = pd.DataFrame(data = sol, columns = ["label"])
sol['filename'] = names_X
sol.to_csv("soln1.csv")

df=pd.read_csv('content/pneumonia_dataset/sample_submission.csv')
df2=pd.read_csv('soln1.csv')
df.drop('label',axis=1,inplace=True)
f3 = df[["filename"]].merge(df2[["filename", "label"]], on = "filename", how = "left")
f3.to_csv("Results.csv", index = False)
os.remove("soln1.csv")

In [ ]:
ef vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

          x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)


          for layer in model.layers[:8]:

       layer.trainable = False

    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model
